In [1]:
import sys; sys.path.insert(0, "..")

import aoc

year, day = 2018, 23

puzzle = aoc.setup(year, day)
plines = puzzle.splitlines()

# Day 23

### Puzzle 1

In [2]:
def solve1():
    bots = []
    for pos, r in map(str.split, plines):
        x, y, z = map(int, pos.split("<")[1].strip(">,").split(","))
        r = int(r.split("=")[1])
        bots.append((x, y, z, r))
    
    sx, sy, sz, sr = max(bots, key=lambda a: a[3])
    return sum(abs(x-sx)+abs(y-sy)+abs(z-sz) <= sr for x, y, z, _ in bots)

solve1()

730

In [3]:
%timeit solve1()

2.38 ms ± 225 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Puzzle 2

In [4]:
from z3 import *

def solve2():
    bots = []
    for pos, r in map(str.split, plines):
        x, y, z = map(int, pos.split("<")[1].strip(">,").split(","))
        r = int(r.split("=")[1])
        bots.append((x, y, z, r))

    zabs = lambda x: If(x >= 0, x, -x)

    x, y, z = Ints("x y z")
    in_ranges = [Int(f"in_range_{i}") for i in range(len(bots))]
    range_count = Int("sum")
    
    o = Optimize()
    for (nx, ny, nz, nr), ir in zip(bots, in_ranges):
        o.add(ir == If(zabs(x - nx) + zabs(y - ny) + zabs(z - nz) <= nr, 1, 0))

    o.add(range_count == sum(in_ranges))
    
    dist_from_zero = Int("dist")
    o.add(dist_from_zero == zabs(x) + zabs(y) + zabs(z))
    
    h1 = o.maximize(range_count)
    h2 = o.minimize(dist_from_zero)

    o.check()
    return o.lower(h2)

solve2()

48202279

In [5]:
%timeit solve2()

1min 31s ± 15.4 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
